In [25]:
###20Newsgroup data set
##Dataset is downloaded from sklearn 
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', shuffle = True)
newsgroups_test = fetch_20newsgroups(subset='test', shuffle = True)

In [26]:
### Let's check what's inside datasets
print(list(newsgroups_train.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [27]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

import pandas as pd
stemmer = SnowballStemmer("english")

go


In [28]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\14373\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [29]:
### Lemmatization and Stemming word
### Both process convert a word into it's basic form.
###

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [30]:
processed_docs = []

for doc in newsgroups_train.data:
    processed_docs.append(preprocess(doc))

In [31]:
### Check processed_docs
print(processed_docs[:2])

[['lerxst', 'thing', 'subject', 'nntp', 'post', 'host', 'organ', 'univers', 'maryland', 'colleg', 'park', 'line', 'wonder', 'enlighten', 'door', 'sport', 'look', 'late', 'earli', 'call', 'bricklin', 'door', 'small', 'addit', 'bumper', 'separ', 'rest', 'bodi', 'know', 'tellm', 'model', 'engin', 'spec', 'year', 'product', 'histori', 'info', 'funki', 'look', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst'], ['guykuo', 'carson', 'washington', 'subject', 'clock', 'poll', 'final', 'summari', 'final', 'clock', 'report', 'keyword', 'acceler', 'clock', 'upgrad', 'articl', 'shelley', 'qvfo', 'innc', 'organ', 'univers', 'washington', 'line', 'nntp', 'post', 'host', 'carson', 'washington', 'fair', 'number', 'brave', 'soul', 'upgrad', 'clock', 'oscil', 'share', 'experi', 'poll', 'send', 'brief', 'messag', 'detail', 'experi', 'procedur', 'speed', 'attain', 'rat', 'speed', 'card', 'adapt', 'heat', 'sink', 'hour', 'usag', 'floppi', 'disk', 'function', 'floppi', 'especi', 'request', 'summar', 'day',

In [32]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [33]:
###Checking dictionary created

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 addit
1 bodi
2 bricklin
3 bring
4 bumper
5 call
6 colleg
7 door
8 earli
9 engin
10 enlighten


In [34]:
##OPTIONAL STEP
##Remove very rare and very common words:
# words appearing less than 15 times
#words appearing in more than 10% of all documents

dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

In [35]:
###Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
##words and how many times those words appear. Save this to 'bow_corpus'

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [36]:
##Preview BOW for our sample preprocessed document
document_num = 20
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 18 ("rest") appears 1 time.
Word 166 ("clear") appears 1 time.
Word 336 ("refer") appears 1 time.
Word 350 ("true") appears 1 time.
Word 391 ("technolog") appears 1 time.
Word 437 ("christian") appears 1 time.
Word 453 ("exampl") appears 1 time.
Word 476 ("jew") appears 1 time.
Word 480 ("lead") appears 1 time.
Word 482 ("littl") appears 3 time.
Word 520 ("wors") appears 2 time.
Word 721 ("keith") appears 3 time.
Word 732 ("punish") appears 1 time.
Word 803 ("california") appears 1 time.
Word 859 ("institut") appears 1 time.
Word 917 ("similar") appears 1 time.
Word 990 ("allan") appears 1 time.
Word 991 ("anti") appears 1 time.
Word 992 ("arriv") appears 1 time.
Word 993 ("austria") appears 1 time.
Word 994 ("caltech") appears 2 time.
Word 995 ("distinguish") appears 1 time.
Word 996 ("german") appears 1 time.
Word 997 ("germani") appears 3 time.
Word 998 ("hitler") appears 1 time.
Word 999 ("livesey") appears 2 time.
Word 1000 ("motto") appears 2 time.
Word 1001 ("order") appear

In [38]:
# LDA multicore 
###Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'

lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 20)

In [39]:
###For each topic, we will explore the words occuring in that topic and its relative weight

for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.015*"window" + 0.005*"wire" + 0.004*"power" + 0.004*"mous" + 0.004*"driver" + 0.003*"drive" + 0.003*"font" + 0.003*"card" + 0.003*"connect" + 0.003*"program"


Topic: 1 
Words: 0.012*"armenian" + 0.007*"turkish" + 0.006*"govern" + 0.006*"israel" + 0.006*"isra" + 0.005*"kill" + 0.005*"jew" + 0.004*"arab" + 0.004*"turk" + 0.004*"armenia"


Topic: 2 
Words: 0.007*"christian" + 0.005*"encrypt" + 0.004*"entri" + 0.004*"file" + 0.004*"truth" + 0.004*"islam" + 0.004*"religion" + 0.004*"chip" + 0.003*"claim" + 0.003*"true"


Topic: 3 
Words: 0.005*"jesus" + 0.004*"live" + 0.003*"life" + 0.003*"kill" + 0.003*"christian" + 0.003*"happen" + 0.003*"caus" + 0.002*"gun" + 0.002*"group" + 0.002*"talk"


Topic: 4 
Words: 0.012*"file" + 0.008*"drive" + 0.007*"program" + 0.007*"window" + 0.006*"softwar" + 0.006*"avail" + 0.005*"version" + 0.005*"imag" + 0.005*"graphic" + 0.005*"data"


Topic: 5 
Words: 0.008*"govern" + 0.005*"encrypt" + 0.005*"space" + 0.005*"clipper" + 0.004*"chip" +

In [ ]:
What we can infer from the above result:- 

0: Graphics Cards
1: Religion
2: Space
3: Politics
4: Gun Violence
5: Technology
6: Sports
7: Encryption